In [112]:
import sys
import logging
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger()
import tqdm
from pathlib import Path

In [113]:
import gspread
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

In [114]:
import pandas as pd

In [115]:
gc = gspread.oauth(http_client=gspread.BackOffHTTPClient)

In [7]:
def duplicate_spreadsheet(gc, spread_sheet, new_file_name, folder_id):
    if matches:=gc.list_spreadsheet_files(new_file_name, folder_id):
        logger.info(f"{new_file_name} exists")
        try:
            return gc.open(matches[0]["id"])
        except:
            pass

    new_spread_sheet = gc.copy(
        file_id=spread_sheet.id,
        title=new_file_name,
        folder_id=folder_id,
    )
    
    return new_spread_sheet

In [8]:
def get_base_wsheet(spread_sheet, wsheet_name):
    for ws in spread_sheet.worksheets():
        if ws.title == wsheet_name:
            return ws
    return None

In [9]:
def copy_base_spread_sheet(base_spread_sheet_url, new_file_name, directory_id):
    base_spreadsheet = gc.open_by_url(base_spread_sheet_url)
    new_sp = duplicate_spreadsheet(gc, base_spreadsheet, new_file_name, directory_id)
    return new_sp

In [59]:
def load_df(path):
    df = (pd
        .read_csv(path, index_col=[0, 1, 2, 3])
        .sample(frac=1)
    )
    return df

In [110]:
def fill_spread_sheet(spread_sheet, df):
    base_sheet = get_base_wsheet(spread_sheet, "blank")
    assert base_sheet is not None
    base_sheet_id = base_sheet.id
    for i, (idx, row) in tqdm.tqdm(enumerate(df.iterrows())):
        gen_id = idx[0]
        source = row["inputs"]
        pred = row["value"]

        ws = spread_sheet.duplicate_sheet(
            source_sheet_id = base_sheet_id,
            insert_sheet_index=i+1,
            new_sheet_name=f"{i+1}"
        )
        ws.update_acell("a1", gen_id)
        ws.update_acell("d1", source)
        ws.update_acell("g1", pred)

            
    spread_sheet.del_worksheet(spread_sheet.get_worksheet(0))

In [111]:
# TGT_FOLDER_ID = "1pV0dYERl9N0y2vD3M8hLTD7Juyi3fqVu"
# BASE_SPREAD_SHEET_URL = "https://docs.google.com/spreadsheets/d/13rSZKWu9oc3nBqz79TgupvoaBGaRpwTT08flCAKII-Q"
# for f in Path("./data/annot_dfs/").glob("samples_ann_*.csv"):
#     name = f"falc.mtl.{int(f.stem.split("_")[-1])}"
#     df = load_df(f)
#     spread_sheet = copy_base_spread_sheet(BASE_SPREAD_SHEET_URL, name, TGT_FOLDER_ID)
#     fill_spread_sheet(spread_sheet, df)

INFO:root:falc.mtl.1 exists


24it [00:44,  1.86s/it]


INFO:root:falc.mtl.2 exists


24it [01:31,  3.81s/it]


INFO:root:falc.mtl.3 exists


...

24it [02:00,  5.01s/it]


INFO:root:falc.mtl.9 exists


24it [02:35,  6.48s/it]


INFO:root:falc.mtl.10 exists


24it [00:51,  2.14s/it]


In [128]:
TGT_FOLDER_ID = "1pV0dYERl9N0y2vD3M8hLTD7Juyi3fqVu"
BASE_SPREAD_SHEET_URL = "https://docs.google.com/spreadsheets/d/13rSZKWu9oc3nBqz79TgupvoaBGaRpwTT08flCAKII-Q"
f = Path("./data/annot_dfs/inter_annotator.csv")
df = load_df(f)
for i in range(10):
    name = f"falc.mtl.interannot.{i}"
    spread_sheet = copy_base_spread_sheet(BASE_SPREAD_SHEET_URL, name, TGT_FOLDER_ID)
    fill_spread_sheet(spread_sheet, df.sample(frac=1))

6it [00:08,  1.49s/it]
6it [00:07,  1.20s/it]
6it [00:39,  6.51s/it]
6it [00:08,  1.38s/it]
6it [00:05,  1.04it/s]
6it [00:38,  6.42s/it]
6it [00:07,  1.24s/it]
6it [00:08,  1.42s/it]
6it [00:38,  6.40s/it]
6it [00:06,  1.04s/it]


In [158]:
df = load_df(f)

In [185]:
df.sample(frac=1)

,,,,inputs,value
level_0,model,test_set,input_id,,
185,labels,test_etr_fr_politic,13,UNE PRIORITÉ : LE CLIMAT. • Plus un euro d’arg...,Protéger la planète. • L’Etat ne donnera plus ...
1153,"('llamaVIIIinst', 'rag', 'etrfr')",test_etr_fr,2,Lucas prend son visage entre ses mains et réfl...,Lucas est inquiet pour Manon. Il demande conse...
1620,"('llamaVIII', 'mtllora', 'etrfr+orangesum+wikilarge')",test_etr_fr,39,Il avale ses tartines à toute vitesse et il va...,Jules mange vite ses tartines. Puis il va voir...
1569,"('llamaVIII', 'mtllora', 'etrfr+orangesum+wikilarge')",test_etr_fr_politic,21,"VIVRE BIEN. Vivre en bonne santé, c’est respir...","VIVRE BIEN. Vivre en bonne santé, c'est respir..."
1122,"('llamaVIIIinst', 'rag', 'etrfr')",test_etr_fr_politic,4,"HÔPITAUX, EHPAD: À BAS LA LOI DU PROFIT! 100 0...",Les hôpitaux et les maisons de retraite sont e...
245,labels,test_etr_fr,40,"Malgré cette situation incroyable, il ouvre la...","« Mais, mais tu es un chat qui parle? » « Oui,..."
